In [8]:
%%capture
!pip install nltk
!pip install ipynb
!pip install s3fs

In [ ]:
import numpy as np
import pandas as pd
import nltk
import sys
from ipynb.fs.full.eval_metrics import *
import json
import s3fs
from functools import reduce

In [ ]:
# Opening JSON file
with open('../credentials.json', 'r') as openfile:
 
    # Reading from json file
    json_object = json.load(openfile)
    key = json_object["key"]
    secret = json_object["secret_key"]
    bucket_name = json_object["bucket_name"]

s3 = s3fs.S3FileSystem(anon=False,key=key,secret=secret)

In [9]:
import numpy as np
import pandas as pd
import nltk
import sys
from ipynb.fs.full.eval_metrics import *
import json
import s3fs
from functools import reduce

In [19]:
def preProcess(tweet):
    tokens = nltk.word_tokenize(tweet)
    tokens = [w.lower() for w in tokens]
    tokens = [w for w in tokens if w.isalpha()]
    return tokens

In [20]:
def tweetToFeatures(tokensList):
    fdist = nltk.FreqDist(tokensList)
    dictionary = dict(fdist)
    s = pd.Series(dictionary)
    return s

In [21]:
tokens = preProcess(data["Text"][1])

In [22]:
def filter(tokens):
    return [w for w in tokens if w.isalpha()]

In [23]:
tokens = filter(tokens)

In [24]:
features = tweetToFeatures(tokens)

In [25]:
bagOfWordsPerTweet = []
for tweet in data["Text"]:
    tokens = preProcess(tweet)
    bagOfWords = tweetToFeatures(tokens)
    bagOfWordsPerTweet.append(bagOfWords)

In [26]:
bagOfWords = pd.DataFrame(bagOfWordsPerTweet)

In [27]:
bagOfWords = bagOfWords.fillna(0)

In [28]:
def filterUniqueWords(featureSpace):
        toDelete = []
        newFs = featureSpace.copy()
        for col in newFs:
            if(sum(newFs[col]) == 1.0):
                toDelete.append(col)
        for word,value in newFs.astype(bool).sum(axis=0).iteritems():
            if(value == 1):
                toDelete.append(word)
        newFs = newFs.drop(list(set(toDelete)), axis=1)
        return newFs

In [29]:
bagOfWords = filterUniqueWords(bagOfWords)

In [30]:
vectors = bagOfWords.values
vectors

array([[1., 1., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [31]:
from sklearn.model_selection import train_test_split

In [32]:
x_train, x_test, y_train, y_test = train_test_split(vectors, data['majority_vote'], test_size=0.2, random_state = 1)

<h1>Naive Bayes<h1>

In [33]:
from sklearn.naive_bayes import MultinomialNB

In [34]:
clf = MultinomialNB().fit(x_train, y_train)

In [35]:
from sklearn.metrics import make_scorer, accuracy_score, balanced_accuracy_score, f1_score, matthews_corrcoef, precision_score, recall_score

In [36]:
scoring = [make_scorer(accuracy_score),
           make_scorer(balanced_accuracy_score),
           make_scorer(f1_score, average='macro'),
           make_scorer(f1_score, average='micro'),
           make_scorer(f1_score, average=None, labels = [1]),
           make_scorer(f1_score, average=None, labels = [2]),
           make_scorer(f1_score, average=None, labels = [3]),
           make_scorer(f1_score, average='weighted'),
           make_scorer(matthews_corrcoef),
           make_scorer(precision_score, average=None, labels = [1]),
           make_scorer(precision_score, average=None, labels = [2]),
           make_scorer(precision_score, average=None, labels = [3]),
           make_scorer(precision_score, average='macro'),
           make_scorer(precision_score, average='micro'),
           make_scorer(precision_score, average='weighted'),
           make_scorer(recall_score, average=None, labels = [1]),
           make_scorer(recall_score, average=None, labels = [2]),
           make_scorer(recall_score, average=None, labels = [3]),
           make_scorer(recall_score, average='macro'),
           make_scorer(recall_score, average='micro'),
           make_scorer(recall_score, average='weighted')
]

In [37]:
from sklearn.model_selection import StratifiedShuffleSplit

In [38]:
sss = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)

In [39]:
sss

StratifiedShuffleSplit(n_splits=5, random_state=42, test_size=0.2,
            train_size=None)

In [40]:
from sklearn.model_selection import cross_val_score

In [43]:
def report(scoring):
    for score in scoring:
        scores = cross_val_score(clf, vectors, data['majority_vote'], scoring=score, cv = sss)
        print(scores.mean())

In [44]:
report(scoring)

0.7310344827586207
0.6781944444444445
0.6802664647273533
0.7310344827586207
0.8075181954292117
0.0


/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1570: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1570: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1570: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_cl

0.0
0.7285343625797823
0.36175784133255184
0.7980889746173032


/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined a

0.0


/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined a

0.0
0.6836790164593296
0.7310344827586207
0.7270758971399401
0.8175000000000001


/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to 

0.0
0.0
0.6781944444444445
0.7310344827586207
0.7310344827586207


<h1>SVM<h1>

In [45]:
from sklearn import svm

In [46]:
clf = svm.SVC()

In [47]:
clf.fit(x_train, y_train)

SVC()

In [48]:
report(scoring)

0.6956896551724138
0.5295833333333333
0.48437300499443925
0.6956896551724139
0.814389323985606


/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1570: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1570: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1570: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_cl

0.0


/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1570: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1570: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1570: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_cl

0.0
0.6095516087497094
0.11375759969301968
0.7031866748269591


...

0.0
0.6104317212518634
0.6956896551724138
0.6456146346768997
0.9675


/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to 

0.0


/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to 

0.0
0.5295833333333333
0.6956896551724138
0.6956896551724138


<h1>Pipeline for comparing methods<h1>

In [49]:
df = data

In [50]:
from sklearn.feature_extraction.text import CountVectorizer

In [51]:
from sklearn.preprocessing import MinMaxScaler

In [52]:
scaler = MinMaxScaler()

In [53]:
from sklearn.ensemble import RandomForestClassifier

In [54]:
from sklearn.linear_model import SGDClassifier

In [55]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [52]:
%%capture
!pip install spacy
!python -m spacy download de

In [56]:
import spacy
from spacy.lang.de.examples import sentences 

In [57]:
nlp = spacy.load('de_core_news_sm')

In [58]:
from pprint import pprint

In [59]:
def lemmatize(text):
    doc = nlp(text)
    return [word.lemma_ for word in doc]

In [60]:
normalizationMethods = ["None", "lower", "lemmatization", "lemmatizationAndLower"]
def normalize(tweetText, approach):
    if(approach == "None"):
        tokens = nltk.word_tokenize(tweetText)
        return tokens
    elif(approach == "lower"):
        tokens = nltk.word_tokenize(tweetText)
        tokens = [w.lower() for w in tokens]
        return tokens
    elif(approach == "lemmatization"):
        doc = nlp(tweetText)
        tokens = lemmatize(tweetText)
        return tokens
    elif(approach == "lemmatizationAndLower"):
        doc = nlp(tweetText)
        tokens = lemmatize(tweetText)
        tokens = [w.lower() for w in tokens]
        return tokens

In [61]:
filterMethods = ["None", "punctuation", "punctuationAndStopwords"]
def filter(tokens, approach):
    if(approach == "None"):
        return tokens
    elif(approach == "punctuation"):
        return [w for w in tokens if w.isalpha()]
    elif(approach == "punctuationAndStopwords"):
        return [w for w in tokens if w.isalpha() and w.lower()]

In [62]:
featureMethods = ["tf"]
def textToFeatures(tokensList, approach):
    if(approach == "tf"):
        fdist = nltk.FreqDist(tokensList)
        dictionary = dict(fdist)
        s = pd.Series(dictionary)
        return s

        

In [63]:
featureSpaceFilterMethods = ["None", "uniqueWords"]
def filterFs(featureSpace, filterMethod):
    if(filterMethod == "None"):
        return featureSpace
    elif(filterMethod == "uniqueWords"):
        toDelete = []
        newFs = featureSpace.copy()
        for col in newFs:
            if(sum(newFs[col]) == 1.0):
                toDelete.append(col)
        for word,value in newFs.astype(bool).sum(axis=0).iteritems():
            if(value == 1):
                toDelete.append(word)
        newFs = newFs.drop(list(set(toDelete)), axis=1)
        return newFs

In [64]:
featureSpaceTransformationMethods = ["None"]
def transformFeatureSpace(featureSpace, transformationMethod):
    if(transformationMethod == "None"):
        return featureSpace
        

In [66]:
def buildFeatureSpace(normalizationApproach, filterApproach, featureMethod, filterFeatureSpace):
    featuresPerTweet = []
    for tweet in df["Text"]:
        tokens = normalize(tweet, normalizationApproach)
        tokens = filter(tokens, filterApproach)
        features = textToFeatures(tokens, featureMethod)
        featuresPerTweet.append(features)
    featureSpace = pd.DataFrame(featuresPerTweet)
    featureSpace = featureSpace.fillna(0)
    featureSpace = filterFs(featureSpace, filterFeatureSpace)
    return featureSpace

In [71]:
classifiers = ["bayes", "svm", "sgdc", "randomForest"]
def classifyAndEvaluate(featureSpace, classifier):
    sss = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
    clf = None
    if(classifier == "bayes"):
        clf = MultinomialNB()
    elif(classifier == "svm"):
        clf = svm.SVC(gamma="scale")
    elif(classifier == "sgdc"):
        clf = SGDClassifier(loss="hinge", penalty="l2", max_iter=5)
    elif(classifier == "randomForest"):
        clf = RandomForestClassifier(n_estimators=100, random_state=42)
    vectors = featureSpace.values
    annotation = df["majority_vote"]
    scores = cross_val_score(clf, vectors, annotation, cv=sss)
    print(scores.mean())
    return scores

In [72]:
def getScores(normalizationApproach, filterApproach, featureMethod, filterFeatureSpace, classifier):
    featureSpace = buildFeatureSpace(normalizationApproach, filterApproach, featureMethod, filterFeatureSpace)
    scores = classifyAndEvaluate(featureSpace, classifier)
    return scores

In [73]:
def compareAndPrint():
    print("\t".join(["Normalization", "Filtering", "Fspace", "Fspace-Filter", "Classifier", "Precision"]))
    for normalization in normalizationMethods:
        for filtering in filterMethods:
            for fspace in featureMethods:
                for fspaceFilter in featureSpaceFilterMethods:
                    for classifier in classifiers:
                        scores = getScores(normalization, filtering, fspace, fspaceFilter, classifier)
                        precision = scores.mean()
                        print("\t".join([normalization, filtering, fspace, fspaceFilter, classifier, str(precision)]))

In [ ]:
compareAndPrint()

Normalization	Filtering	Fspace	Fspace-Filter	Classifier	Precision
0.7318965517241379
None	None	tf	None	bayes	0.7318965517241379
0.6965517241379311
None	None	tf	None	svm	0.6965517241379311


/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_grad

0.6948275862068967
None	None	tf	None	sgdc	0.6948275862068967
0.6922413793103448
None	None	tf	None	randomForest	0.6922413793103448
0.7405172413793102
None	None	tf	uniqueWords	bayes	0.7405172413793102
0.6974137931034482
None	None	tf	uniqueWords	svm	0.6974137931034482


/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_grad

0.6991379310344829
None	None	tf	uniqueWords	sgdc	0.6991379310344829
0.7
None	None	tf	uniqueWords	randomForest	0.7
0.7241379310344828
None	punctuation	tf	None	bayes	0.7241379310344828
0.6844827586206896
None	punctuation	tf	None	svm	0.6844827586206896


...

0.6905172413793104
lower	punctuationAndStopwords	tf	None	sgdc	0.6905172413793104
0.7008620689655173
lower	punctuationAndStopwords	tf	None	randomForest	0.7008620689655173
0.7310344827586207
lower	punctuationAndStopwords	tf	uniqueWords	bayes	0.7310344827586207
0.6956896551724138
lower	punctuationAndStopwords	tf	uniqueWords	svm	0.6956896551724138


/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_grad

0.6887931034482759
lower	punctuationAndStopwords	tf	uniqueWords	sgdc	0.6887931034482759
0.7086206896551724
lower	punctuationAndStopwords	tf	uniqueWords	randomForest	0.7086206896551724
0.7353448275862069
lemmatization	None	tf	None	bayes	0.7353448275862069
0.6956896551724137
lemmatization	None	tf	None	svm	0.6956896551724137


/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_grad

0.6905172413793104
lemmatization	None	tf	None	sgdc	0.6905172413793104


# Building SVM Model

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm

def my_custom_metric(y_true, y_pred):
    test_list = y_true
    # Compute Evaluation Metrics
    f = f1_multiclass(test_list, y_pred)
    p = p_multiclass(test_list, y_pred)
    r = recall_multiclass(test_list, y_pred)
    a = accuracy_score(test_list, y_pred)
    ba = balanced_accuracy_score(test_list, y_pred)
    prs = precision_recall_fscore_support(test_list, y_pred)
    m = matthews_corrcoef(test_list,y_pred)

    results = {}
    results["acc"] = a
    results["f1"] = f
    results["precision"] = p
    results["recall"] = r
    results["bal_acc"] = ba
    results["prfs"] = prs
    results["mcc"] = m
    return results

def fold_values(fold_f, fold_number):
    folds = pd.DataFrame(columns=["Model", "F1_Score"])
    models = []
    f1s = []
    p = []
    r = []

    for i in kfold_df[f"model_fold_{fold_number}"]:
        name = i.get("name")
        f1 = i.get("custom_metric_score").get("f1").get("f1_weighted")
        pr = i.get("custom_metric_score").get("precision").get("precision_weighted")
        re = i.get("custom_metric_score").get("recall").get("recall_weighted")
        models.append(name)
        f1s.append(f1)
        p.append(pr)
        r.append(re)


    folds["Model"] = models
    folds["F1_Score"] = f1s
    folds["Precision"] = p
    folds["Recall"] = r
    return folds

In [39]:
# import re
from sklearn import metrics

train_test_number = [1,2,3,4,5]
train_file_name = "train.csv"
test_file_name = "test.csv"
kfold_df = pd.DataFrame()

for train_index in train_test_number:
    print(f"Fold Number: {train_index}")
    # Read data
    with s3.open(f"{bucket_name}/KFOLD{train_index}/{train_file_name}",'r') as file:
        data = pd.read_csv(file)
    with s3.open(f"{bucket_name}/KFOLD{train_index}/{test_file_name}",'r') as file:
        test_data = pd.read_csv(file)

    train_text = data['Text']
    train_labels = data['majority_vote']
    test_text = test_data['Text']
    test_labels = test_data['majority_vote']
    
    # Tokenize the words
    train_text_clean = train_text.apply(nltk.word_tokenize)
    test_text_clean = test_text.apply(nltk.word_tokenize)

    # Remove stop words
    stop_words=set(nltk.corpus.stopwords.words("german"))
    train_text_clean = train_text_clean.apply(lambda x: [item for item in x if item not in stop_words])
    test_text_clean = test_text_clean.apply(lambda x: [item for item in x if item not in stop_words])

    # Remove numbers, punctuation and special characters (only keep words)
    regex = '[a-z]+'
    train_text_clean = train_text_clean.apply(lambda x: [item for item in x if re.match(regex, item)])
    test_text_clean = test_text_clean.apply(lambda x: [item for item in x if re.match(regex, item)])

    # Lemmatization
    lem = nltk.stem.wordnet.WordNetLemmatizer()
    train_text_clean = train_text_clean.apply(lambda x: [lem.lemmatize(item, pos='v') for item in x])
    test_text_clean = test_text_clean.apply(lambda x: [lem.lemmatize(item, pos='v') for item in x])

    # Join the words again to form sentences
    train_text_clean = train_text_clean.apply(lambda x: " ".join(x))
    test_text_clean = test_text_clean.apply(lambda x: " ".join(x))
    
    # Tfidf vectorization
    vectorizer = TfidfVectorizer()

    X_train = vectorizer.fit_transform(train_text)
    X_test = vectorizer.transform(test_text)
    y_train = train_labels
    y_test = test_labels
    
    clf = svm.SVC(kernel = "linear")
    # Training the classifier
    clf_trained = clf.fit(X_train,y_train)
    # Scoring the classifier
    clf_trained.score(X_test,y_test)

    #extract the predictions of the model
    test_pred_svm = clf_trained.predict(X_test)
    #print the classification report
    print (metrics.classification_report(y_test, test_pred_svm))
    test = my_custom_metric(y_test, test_pred_svm) 
    test_ = pd.DataFrame([test])
    #kfold_df = kfold_df.append(test_)
    test_.to_csv(f"SVM_KFOLD_{train_index}.csv")

Fold Number: 1
              precision    recall  f1-score   support

           0       0.56      0.32      0.41        72
           1       0.74      0.89      0.81       160

    accuracy                           0.71       232
   macro avg       0.65      0.60      0.61       232
weighted avg       0.69      0.71      0.68       232

Fold Number: 2


/tmp/ipykernel_559/300311922.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  kfold_df = kfold_df.append(test_)


              precision    recall  f1-score   support

           0       0.57      0.29      0.39        72
           1       0.74      0.90      0.81       160

    accuracy                           0.71       232
   macro avg       0.65      0.60      0.60       232
weighted avg       0.69      0.71      0.68       232

Fold Number: 3


/tmp/ipykernel_559/300311922.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  kfold_df = kfold_df.append(test_)


              precision    recall  f1-score   support

           0       0.68      0.39      0.50        72
           1       0.77      0.92      0.84       160

    accuracy                           0.75       232
   macro avg       0.73      0.65      0.67       232
weighted avg       0.74      0.75      0.73       232

Fold Number: 4


/tmp/ipykernel_559/300311922.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  kfold_df = kfold_df.append(test_)


              precision    recall  f1-score   support

           0       0.65      0.31      0.42        71
           1       0.75      0.93      0.83       160

    accuracy                           0.74       231
   macro avg       0.70      0.62      0.62       231
weighted avg       0.72      0.74      0.70       231

Fold Number: 5


/tmp/ipykernel_559/300311922.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  kfold_df = kfold_df.append(test_)


              precision    recall  f1-score   support

           0       0.67      0.39      0.49        72
           1       0.77      0.91      0.83       159

    accuracy                           0.75       231
   macro avg       0.72      0.65      0.66       231
weighted avg       0.74      0.75      0.73       231



/tmp/ipykernel_559/300311922.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  kfold_df = kfold_df.append(test_)


In [40]:
kfold_df = kfold_df.reset_index()

In [41]:
# Compute Result metrics

f1_micro_l = []
f1_macro_l = []
f1_w_l = []

p_micro_l = []
p_macro_l = []
p_w_l = []

r_micro_l = []
r_macro_l = []
r_w_l = []

for index, row in kfold_df.iterrows():
    f1_micro = row["f1"].get("f1_micro")
    f1_micro_l.append(f1_micro)
    f1_macro = row["f1"].get("f1_macro")
    f1_macro_l.append(f1_macro)
    f1_w = row["f1"].get("f1_weighted")
    f1_w_l.append(f1_w)
    
    p_micro = row["precision"].get("precision_micro")
    p_micro_l.append(p_micro)
    p_macro = row["precision"].get("precision_macro")
    p_macro_l.append(p_macro)
    p_w = row["precision"].get("precision_weighted")
    p_w_l.append(p_w)
    
    r_micro = row["recall"].get("recall_micro")
    r_micro_l.append(r_micro)
    r_macro = row["recall"].get("recall_macro")
    r_macro_l.append(r_macro)
    r_w = row["recall"].get("recall_weighted")
    r_w_l.append(r_w)
    
kfold_df["f1_micro"] = f1_micro_l
kfold_df["f1_macro"] = f1_macro_l
kfold_df["f1_weighted"] = f1_w_l
kfold_df = kfold_df.drop("f1", axis=1)
kfold_df["precision_micro"] = p_micro_l
kfold_df["precision_macro"] = p_macro_l
kfold_df["precision_weighted"] = p_w_l
kfold_df = kfold_df.drop("precision", axis=1)
kfold_df["recall_micro"] = r_micro_l
kfold_df["recall_macro"] = r_macro_l
kfold_df["recall_weighted"] = r_w_l
kfold_df = kfold_df.drop("recall", axis=1)
kfold_df

,index,acc,bal_acc,prfs,mcc,f1_micro,f1_macro,f1_weighted,precision_micro,precision_macro,precision_weighted,recall_micro,recall_macro,recall_weighted
0,0,0.711207,0.603472,"([0.5609756097560976, 0.743455497382199], [0.3...",0.250999,0.711207,0.608098,0.684347,0.711207,0.652216,0.686824,0.711207,0.603472,0.711207
1,0,0.711207,0.595833,"([0.5675675675675675, 0.7384615384615385], [0....",0.242189,0.711207,0.598294,0.679077,0.711207,0.653015,0.685425,0.711207,0.595833,0.711207
2,0,0.754310,0.653819,"([0.6829268292682927, 0.7696335078534031], [0....",0.373129,0.754310,0.666591,0.731459,0.754310,0.726280,0.742725,0.754310,0.653819,0.754310
3,0,0.735931,0.617430,"([0.6470588235294118, 0.751269035532995], [0.3...",0.305861,0.735931,0.624090,0.703089,0.735931,0.699164,0.719239,0.735931,0.617430,0.735931
4,0,0.748918,0.650419,"([0.6666666666666666, 0.7671957671957672], [0....",0.361279,0.748918,0.662281,0.726703,0.748918,0.716931,0.735862,0.748918,0.650419,0.748918


In [43]:
kfold_df.loc["Total"] = kfold_df.sum()

In [53]:
# Results of the SVM model after 5-Fold Cross Validation:
print(f"Acc: {kfold_df['acc']['Total'] / 5}")
print(f"F1 Micro: {kfold_df['f1_micro']['Total'] / 5}")
print(f"F1 Macro: {kfold_df['f1_macro']['Total'] / 5}")
print(f"F1 Weighted: {kfold_df['f1_weighted']['Total'] / 5}")
print(f"Precision Micro: {kfold_df['precision_micro']['Total'] / 5}")
print(f"Precision Macro: {kfold_df['precision_macro']['Total'] / 5}")
print(f"Precision Weighted: {kfold_df['precision_weighted']['Total'] / 5}")
print(f"Recall Micro: {kfold_df['recall_micro']['Total'] / 5}")
print(f"Recall Macro: {kfold_df['recall_macro']['Total'] / 5}")
print(f"Recall Weighted: {kfold_df['recall_weighted']['Total'] / 5}")

Acc: 0.7323145245559038
F1 Micro: 0.7323145245559038
F1 Macro: 0.6318707895748974
F1 Weighted: 0.704934938056964
Precision Micro: 0.7323145245559038
Precision Macro: 0.6895210843213938
Precision Weighted: 0.7140149807626603
Recall Micro: 0.7323145245559038
Recall Macro: 0.6241947729353058
Recall Weighted: 0.7323145245559038
